# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8accb1b070>
├── 'a' --> tensor([[-1.4146,  1.4255,  1.9142],
│                   [-0.2560, -0.6304, -1.2516]])
└── 'x' --> <FastTreeValue 0x7f8accb1bb50>
    └── 'c' --> tensor([[ 0.5729,  1.3876,  0.0230, -0.6240],
                        [ 1.2857, -1.4434, -1.2680, -0.3311],
                        [ 0.1471,  0.4810, -0.6271,  0.1074]])

In [4]:
t.a

tensor([[-1.4146,  1.4255,  1.9142],
        [-0.2560, -0.6304, -1.2516]])

In [5]:
%timeit t.a

62.6 ns ± 0.0305 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8accb1b070>
├── 'a' --> tensor([[ 0.2556,  1.7871,  0.0265],
│                   [-1.4001,  0.3829,  1.8526]])
└── 'x' --> <FastTreeValue 0x7f8accb1bb50>
    └── 'c' --> tensor([[ 0.5729,  1.3876,  0.0230, -0.6240],
                        [ 1.2857, -1.4434, -1.2680, -0.3311],
                        [ 0.1471,  0.4810, -0.6271,  0.1074]])

In [7]:
%timeit t.a = new_value

60.7 ns ± 0.0315 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.4146,  1.4255,  1.9142],
               [-0.2560, -0.6304, -1.2516]]),
    x: Batch(
           c: tensor([[ 0.5729,  1.3876,  0.0230, -0.6240],
                      [ 1.2857, -1.4434, -1.2680, -0.3311],
                      [ 0.1471,  0.4810, -0.6271,  0.1074]]),
       ),
)

In [10]:
b.a

tensor([[-1.4146,  1.4255,  1.9142],
        [-0.2560, -0.6304, -1.2516]])

In [11]:
%timeit b.a

59 ns ± 0.109 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.0385, -1.1737, -1.9356],
               [-1.3958,  0.2620, -1.3441]]),
    x: Batch(
           c: tensor([[ 0.5729,  1.3876,  0.0230, -0.6240],
                      [ 1.2857, -1.4434, -1.2680, -0.3311],
                      [ 0.1471,  0.4810, -0.6271,  0.1074]]),
       ),
)

In [13]:
%timeit b.a = new_value

494 ns ± 0.319 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

836 ns ± 1.09 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 50.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 1.09 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

148 µs ± 1.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8accb21310>
├── 'a' --> tensor([[[-1.4146,  1.4255,  1.9142],
│                    [-0.2560, -0.6304, -1.2516]],
│           
│                   [[-1.4146,  1.4255,  1.9142],
│                    [-0.2560, -0.6304, -1.2516]],
│           
│                   [[-1.4146,  1.4255,  1.9142],
│                    [-0.2560, -0.6304, -1.2516]],
│           
│                   [[-1.4146,  1.4255,  1.9142],
│                    [-0.2560, -0.6304, -1.2516]],
│           
│                   [[-1.4146,  1.4255,  1.9142],
│                    [-0.2560, -0.6304, -1.2516]],
│           
│                   [[-1.4146,  1.4255,  1.9142],
│                    [-0.2560, -0.6304, -1.2516]],
│           
│                   [[-1.4146,  1.4255,  1.9142],
│                    [-0.2560, -0.6304, -1.2516]],
│           
│                   [[-1.4146,  1.4255,  1.9142],
│                    [-0.2560, -0.6304, -1.2516]]])
└── 'x' --> <FastTreeValue 0x7f89e51cff40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.8 µs ± 55.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f89e52331f0>
├── 'a' --> tensor([[-1.4146,  1.4255,  1.9142],
│                   [-0.2560, -0.6304, -1.2516],
│                   [-1.4146,  1.4255,  1.9142],
│                   [-0.2560, -0.6304, -1.2516],
│                   [-1.4146,  1.4255,  1.9142],
│                   [-0.2560, -0.6304, -1.2516],
│                   [-1.4146,  1.4255,  1.9142],
│                   [-0.2560, -0.6304, -1.2516],
│                   [-1.4146,  1.4255,  1.9142],
│                   [-0.2560, -0.6304, -1.2516],
│                   [-1.4146,  1.4255,  1.9142],
│                   [-0.2560, -0.6304, -1.2516],
│                   [-1.4146,  1.4255,  1.9142],
│                   [-0.2560, -0.6304, -1.2516],
│                   [-1.4146,  1.4255,  1.9142],
│                   [-0.2560, -0.6304, -1.2516]])
└── 'x' --> <FastTreeValue 0x7f89e5233940>
    └── 'c' --> tensor([[ 0.5729,  1.3876,  0.0230, -0.6240],
                        [ 1.2857, -1.4434, -1.2680, -0.3311],
                 

In [23]:
%timeit t_cat(trees)

30.6 µs ± 74.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

68.9 µs ± 157 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-1.4146,  1.4255,  1.9142],
                [-0.2560, -0.6304, -1.2516]],
       
               [[-1.4146,  1.4255,  1.9142],
                [-0.2560, -0.6304, -1.2516]],
       
               [[-1.4146,  1.4255,  1.9142],
                [-0.2560, -0.6304, -1.2516]],
       
               [[-1.4146,  1.4255,  1.9142],
                [-0.2560, -0.6304, -1.2516]],
       
               [[-1.4146,  1.4255,  1.9142],
                [-0.2560, -0.6304, -1.2516]],
       
               [[-1.4146,  1.4255,  1.9142],
                [-0.2560, -0.6304, -1.2516]],
       
               [[-1.4146,  1.4255,  1.9142],
                [-0.2560, -0.6304, -1.2516]],
       
               [[-1.4146,  1.4255,  1.9142],
                [-0.2560, -0.6304, -1.2516]]]),
    x: Batch(
           c: tensor([[[ 0.5729,  1.3876,  0.0230, -0.6240],
                       [ 1.2857, -1.4434, -1.2680, -0.3311],
                       [ 0.1471,  0.4810, -0.6271,  0.1074]],
         

In [26]:
%timeit Batch.stack(batches)

80.3 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-1.4146,  1.4255,  1.9142],
               [-0.2560, -0.6304, -1.2516],
               [-1.4146,  1.4255,  1.9142],
               [-0.2560, -0.6304, -1.2516],
               [-1.4146,  1.4255,  1.9142],
               [-0.2560, -0.6304, -1.2516],
               [-1.4146,  1.4255,  1.9142],
               [-0.2560, -0.6304, -1.2516],
               [-1.4146,  1.4255,  1.9142],
               [-0.2560, -0.6304, -1.2516],
               [-1.4146,  1.4255,  1.9142],
               [-0.2560, -0.6304, -1.2516],
               [-1.4146,  1.4255,  1.9142],
               [-0.2560, -0.6304, -1.2516],
               [-1.4146,  1.4255,  1.9142],
               [-0.2560, -0.6304, -1.2516]]),
    x: Batch(
           c: tensor([[ 0.5729,  1.3876,  0.0230, -0.6240],
                      [ 1.2857, -1.4434, -1.2680, -0.3311],
                      [ 0.1471,  0.4810, -0.6271,  0.1074],
                      [ 0.5729,  1.3876,  0.0230, -0.6240],
                      [ 1.2857, -

In [28]:
%timeit Batch.cat(batches)

151 µs ± 1.74 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

364 µs ± 2.88 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
